<img src=./imgs/model_io.jpg width=35% />

[langchain documents](https://python.langchain.com/docs/modules/model_io/models/chat/llm_chain)

[LangChain-Tutorials](https://github.com/sugarforever/LangChain-Tutorials)

In [1]:
import os

In [2]:
os.environ['OPENAI_API_KEY'] = '****'

# FewShotPromptTemplate

> `FewShotPromptTemplate`: 可以从**examples set**构建, 也可以从 **Example Selector**

> 例子： 通过搜索自我询问

## 一： example set Implementation

In [1]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

**examples set**

In [2]:
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

### Create a formatter for the few shot examples

In [3]:
example_prompt = PromptTemplate(input_variables=['question', 'answer'], template="Quesion: {question}\n{answer}")

In [5]:
print(example_prompt.format(**examples[0]))

Quesion: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



### Feed examples and formatter to `FewShotPromptTemplate

In [6]:
prompt = FewShotPromptTemplate(examples=examples,
                               example_prompt=example_prompt,
                               suffix='Quesion: {input}',
                               input_variables=['input']
                              )

In [7]:
print(prompt.format(input='Who was the father of Mary Ball Washington?'))

Quesion: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Quesion: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Quesion: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Wash

## 二： example selector implementation

> 重用之前定义的 examples 和 formatter(template)<br>
> 不是直接将examples喂给`FewShotPromptTemplate`, 我们将其喂给`ExampleSelector`

> 这里我们将使用`SemanticSimilarityExampleSelector`: 根据与输入的相似度选择few shot example. <br>
> 它使用embedding模型计算相似度（输入与 examples中的任一样例）， 并借助向量数据库做最近邻查找

In [12]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

### feed examples into `ExampleSelector`

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # this the list of examples available to select from.
    examples=examples, 
    # this is the embedding class used to produce embedding which are used to measure semantic similarity
    embeddings=OpenAIEmbeddings(), 
    # VectorStore clss
    vectorstore_cls=Chroma, 
    # this is the number of example to produce.
    k=1)

In [20]:
# select the most similarity example to the input
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similarity to the input: {question}")
for example in selected_examples:
    print('\n')
    for k, v in example.items():
        print(f'-> {k}: {v}')
    print('---------------example spliter--------------')

Examples most similarity to the input: Who was the father of Mary Ball Washington?


-> answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

-> question: Who was the maternal grandfather of George Washington?
---------------example spliter--------------


### Feed example selector into `FewShotPromptTemplate`

In [22]:
prompt = FewShotPromptTemplate(
                               example_selector=example_selector,
                               example_prompt=example_prompt,
                               suffix='Question: {input}',
                               input_variables=['input']
                              )

VS<br>
```
prompt = FewShotPromptTemplate(examples=examples,
                               example_prompt=example_prompt,
                               suffix='Quesion: {input}',
                               input_variables=['input']
                              )
```

In [23]:
print(prompt.format(input='Who was the father of Mary Ball Washington?'))

Quesion: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


## 三： 结论

1. 基于`FewShotPromptTemplate`的实现，只是格式化了所有 few shot<br>
2. 基于`SemanticSimilarityExampleSelector`的实现，只保留了与问题最相关的 few-shot ， 定点打击，性能应该更好.